In [136]:
from collections import Counter
import numpy as np
import pandas as pd
from scipy.stats.mstats import gmean
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
pd.set_option("display.max_columns",500)
pd.set_option("display.max_rows",50)

In [134]:
# From https://gist.github.com/rspeare/77061e6e317896be29c6de9a85db301d
class LogisticReg:
    """
    Wrapper Class for Logistic Regression which has the usual sklearn instance 
    in an attribute self.model, and pvalues, z scores and estimated 
    errors for each coefficient in 
    
    self.z_scores
    self.p_values
    self.sigma_estimates
    
    as well as the negative hessian of the log Likelihood (Fisher information)
    
    self.F_ij
    """
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)
        #### Get p-values for the fitted model ####
        denom = (2.0*(1.0+np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T  # From rizzomichaelg
        # F_ij = np.dot((X/denom[:,None]).T,X) ## Fisher Information Matrix
        F_ij = np.dot((X/denom).T,X) ## Fisher Information Matrix  # From rizzomichaelg
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.array([np.sqrt(Cramer_Rao[i,i]) for i in range(Cramer_Rao.shape[0])]) # sigma for each coefficient
        z_scores = self.model.coef_[0]/sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values
        
        self.z_scores = z_scores
        print('INFO: Assigning p_values to LogisticReg object')
        self.p_values = p_values
        self.sigma_estimates = sigma_estimates
        self.F_ij = F_ij
        
        alpha = 0.05
        q = stats.norm.ppf(1 - alpha / 2)
        lower = self.model.coef_[0] - q * sigma_estimates
        upper = self.model.coef_[0] + q * sigma_estimates
        self.conf_int = np.dstack((lower, upper))[0]


In [5]:
ARRHYTHMIA_TO_NAME = {1: 'normal'
                      , 2: 'ischemic_changes'
                      , 3: 'old_anterior_myocardial_infarction'
                      , 4: 'old_inferior_myocardial_infarction'
                      , 5: 'sinus_tachycardia'
                      , 6: 'sinus_bradycardia'
                      , 7: 'ventricular_premature_contraction'
                      , 8: 'supraventricular_premature_contraction'
                      , 9: 'left_bundle_branch_block'
                      , 10: 'right_bundle_branch_block'
                      , 11: 'first_degree_atrioventricular_block'
                      , 12: 'second_degree_atrioventricular_block'
                      , 13: 'third_degree_atrioventricular_block'
                      , 14: 'left_ventricule_hypertrophy'
                      , 15: 'atrial_fibrilation_or_flutter'
                      , 16: 'other'
                     }

<hr style="height:5px;margin:auto;width:100%"/>

In [6]:
def arrhythmia_code_to_name(code):
    assert(code in ARRHYTHMIA_TO_NAME.keys())
    return ARRHYTHMIA_TO_NAME[code]

In [7]:
def get_col_names():
    col_names = """
        age sex height weight
        qrs_duration_msec p_onset_to_q_onset_msec
        q_onset_to_t_offset_msec duration_between_t_waves_msec
        duration_between_p_waves_msec
        qrs_front_plane_deg t_front_plane_deg p_front_plane_deg qrst_front_plane_deg
        j_front_plane_deg
        heart_rate_bpm
        """.split()

    t_attrs = """
        q_width_msec
        r_width_msec s_width_msec
        r2_width_msec s2_width_msec
        intrinsic_deflections
        is_present_diphasic_r is_present_notched_r
        is_present_notched_p is_present_diphasic_p
        is_present_notched_t is_present_diphasic_t
        """.split()

    col_names.extend(prefix_list('d1', t_attrs))  # 16-27
    col_names.extend(prefix_list('d2', t_attrs))  # 28-39
    col_names.extend(prefix_list('d3', t_attrs))  # 40-51
    col_names.extend(prefix_list('avr', t_attrs))  # 52-63
    col_names.extend(prefix_list('avl', t_attrs))  # 64-75
    col_names.extend(prefix_list('avf', t_attrs))  # 76-87
    col_names.extend(prefix_list('v1', t_attrs))  # 88-99
    col_names.extend(prefix_list('v2', t_attrs))  # 100-111
    col_names.extend(prefix_list('v3', t_attrs))  # 112-123
    col_names.extend(prefix_list('v4', t_attrs))  # 124-135
    col_names.extend(prefix_list('v5', t_attrs))  # 136-147
    col_names.extend(prefix_list('v6', t_attrs))  # 148-159

    v_attrs = """
        j_point_depression_mvolt q_amplitude_mvolt r_amplitude_mvolt s_amplitude_mvolt
        r2_amplitude_mvolt s2_amplitude_mvolt p_amplitude_mvolt t_amplitude_mvolt
        qrsa, qrsta
        """.split()

    col_names.extend(prefix_list('d1', v_attrs))  # 160-169
    col_names.extend(prefix_list('d2', v_attrs))  # 170-179
    col_names.extend(prefix_list('d3', v_attrs))  # 180-189
    col_names.extend(prefix_list('avr', v_attrs))  # 190-199
    col_names.extend(prefix_list('avl', v_attrs))  # 200-209
    col_names.extend(prefix_list('avf', v_attrs))  # 210-219
    col_names.extend(prefix_list('v1', v_attrs))  # 220-229
    col_names.extend(prefix_list('v2', v_attrs))  # 230-239
    col_names.extend(prefix_list('v3', v_attrs))  # 240-249
    col_names.extend(prefix_list('v4', v_attrs))  # 250-259
    col_names.extend(prefix_list('v5', v_attrs))  # 260-269
    col_names.extend(prefix_list('v6', v_attrs))  # 270-279
    col_names.append('arrhythmia_code')
    return col_names

In [8]:
def prefix_list(pre, items):
    return list(map(lambda x: pre + '_' + x, items))

In [9]:
def nan_percentage(ser):
    return 100.0 * (len(ser) - ser.count()) / len(ser)

<hr style="height:5px;margin:auto;width:100%"/>

In [10]:
cols_with_unknowns = """t_front_plane_deg p_front_plane_deg qrst_front_plane_deg
                        j_front_plane_deg heart_rate_bpm""".split()
cols_dtypes = {col: np.float64 for col in cols_with_unknowns}
uci_df = pd.read_csv('../data/uci.edu/arrhythmia.data.txt', header=None, names=get_col_names()
                      , dtype=cols_dtypes, na_values='?')

In [11]:
uci_df.shape

(452, 280)

In [12]:
for col in cols_with_unknowns:
    print(f'Nan percentage in {col}: {nan_percentage(uci_df[col]):.2f}%')

Nan percentage in t_front_plane_deg: 1.77%
Nan percentage in p_front_plane_deg: 4.87%
Nan percentage in qrst_front_plane_deg: 0.22%
Nan percentage in j_front_plane_deg: 83.19%
Nan percentage in heart_rate_bpm: 0.22%


In [13]:
uci_df.columns[-1]

'arrhythmia_code'

In [14]:
cntr = Counter(uci_df['arrhythmia_code'])

In [15]:
sorted(cntr.items(), key=lambda pair: pair[1], reverse=True)

[(1, 245),
 (10, 50),
 (2, 44),
 (6, 25),
 (16, 22),
 (3, 15),
 (4, 15),
 (5, 13),
 (9, 9),
 (15, 5),
 (14, 4),
 (7, 3),
 (8, 2)]

In [16]:
[(arrhythmia_code_to_name(k), v)
 for k, v in sorted(cntr.items(), key=lambda pair: pair[1], reverse=True)]

[('normal', 245),
 ('right_bundle_branch_block', 50),
 ('ischemic_changes', 44),
 ('sinus_bradycardia', 25),
 ('other', 22),
 ('old_anterior_myocardial_infarction', 15),
 ('old_inferior_myocardial_infarction', 15),
 ('sinus_tachycardia', 13),
 ('left_bundle_branch_block', 9),
 ('atrial_fibrilation_or_flutter', 5),
 ('left_ventricule_hypertrophy', 4),
 ('ventricular_premature_contraction', 3),
 ('supraventricular_premature_contraction', 2)]

In [17]:
missing_arrhythmias = [arrhythmia_code_to_name(code)
                       for code in range(1, 17)
                       if code not in cntr.keys()]
missing_arrhythmias

['first_degree_atrioventricular_block',
 'second_degree_atrioventricular_block',
 'third_degree_atrioventricular_block']

<hr style="height:5px;margin:auto;width:100%"/>

In [18]:
good_codes = [1,10,2,6,16]

In [19]:
uci2_mask = uci_df['arrhythmia_code'].isin(good_codes)
uci2_df = uci_df[uci2_mask].copy()

In [20]:
for col in cols_with_unknowns:
    print(f'Nan percentage in {col}: {nan_percentage(uci2_df[col]):.2f}%')

Nan percentage in t_front_plane_deg: 1.81%
Nan percentage in p_front_plane_deg: 3.89%
Nan percentage in qrst_front_plane_deg: 0.26%
Nan percentage in j_front_plane_deg: 86.01%
Nan percentage in heart_rate_bpm: 0.00%


In [21]:
cols_to_drop = [col for col in cols_with_unknowns if col != 'heart_rate_bpm']
cols_to_drop

['t_front_plane_deg',
 'p_front_plane_deg',
 'qrst_front_plane_deg',
 'j_front_plane_deg']

In [22]:
uci2_df.shape

(386, 280)

In [23]:
X = uci2_df.iloc[:,:-1].copy().drop(cols_to_drop, axis=1)
X = sm.add_constant(X)
y = uci2_df['arrhythmia_code']

In [24]:
X.shape

(386, 276)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345
                                                   , stratify=y)

In [80]:
logreg_cs = [10**k for k in range(-9, 10)]

In [81]:
logreg_cv = LogisticRegressionCV(Cs=tuple(logreg_cs), cv=10, fit_intercept=True, refit=False)
                                 # solver='liblinear', penalty='l1')
logreg_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=(1e-09, 1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000),
           class_weight=None, cv=10, dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=100, multi_class='ovr',
           n_jobs=1, penalty='l1', random_state=None, refit=False,
           scoring=None, solver='liblinear', tol=0.0001, verbose=0)

In [83]:
logreg_cv.coef_

array([[ 0.7165245 ,  0.01551978, -0.4612756 , ..., -0.06384901,
        -0.00620649, -0.01302908],
       [-0.0497906 ,  0.00734883,  0.06961171, ..., -0.31030918,
         0.04009288,  0.01054023],
       [ 0.        , -0.02344039,  0.        , ...,  0.05900708,
        -0.04137017,  0.00132867],
       [-0.21434399, -0.00803146, -0.28165811, ..., -0.04832687,
         0.02229918,  0.01310413],
       [ 0.        ,  0.00384435,  0.        , ...,  0.        ,
        -0.02564413,  0.0205607 ]])

In [113]:
logreg_cv.C_

array([1.00020001e+07, 1.11110001e+06, 1.02400000e+01, 2.10016000e+02,
       1.02000000e+01])

In [86]:
logreg_cv.classes_

array([ 1,  2,  6, 10, 16])

In [103]:
gmean(logreg_cv.C_)

3001.915780104124

<hr style="height:5px;margin:auto;width:50%"/>

In [137]:
y_normal = y.map(lambda x: x if x == 1 else 0)
logreg = LogisticRegression(C=gmean(logreg_cv.C_)
                            , fit_intercept=True, verbose=1)
                            # solver='liblinear', penalty='l1'
logreg.fit(X_test, y_test)
# logreg.predict_proba(X)
y_pred = logreg.predict(X_test)
y_pred

[LibLinear]

array([ 1, 10,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  6, 10,  1,  1,
        2,  2,  6,  1,  1,  1, 16,  1,  1,  1,  6,  2, 10,  1,  1,  1,  2,
        1, 10,  1,  1, 10, 10,  1,  2,  2,  2, 10, 10,  1,  1,  1, 10,  1,
        1,  1, 10,  1, 16,  2,  1, 16,  1,  1,  6,  1, 10,  1,  1,  1,  1,
        1,  2,  1,  1,  1,  6,  1, 10,  1, 10,  6,  1,  1, 16,  1, 16,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2])

In [138]:
report = classification_report(y_test, y_pred
                               , target_names=list(map(arrhythmia_code_to_name, good_codes)))

In [141]:
print(report)

                           precision    recall  f1-score   support

                   normal       1.00      1.00      1.00        62
right_bundle_branch_block       1.00      1.00      1.00        11
         ischemic_changes       1.00      1.00      1.00         6
        sinus_bradycardia       1.00      1.00      1.00        13
                    other       1.00      1.00      1.00         5

              avg / total       1.00      1.00      1.00        97



<hr style="height:5px;margin:auto;width:50%"/>

<hr style="height:5px;margin:auto;width:100%"/>